In [81]:
import redis 

client = redis.Redis('host.docker.internal', 6379, 0)

client.flushdb()

True

In [82]:
client.close()

In [83]:
import json
import numpy as np
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from pcomp.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler, KafkaConsumerHandlerNeuron
from pcomp.activation_functions import ACTIVATIONS, relu, softmax
from pcomp.redis_utils import RedisHandler
from pcomp.neurons_accumulator import NeuronsAccumulator
from pcomp.s3client import S3Client
from io import StringIO

# Kafka Configuration
KAFKA_BROKER = 'kafka:29092'
redis_url = "host.docker.internal:6379"
s3_host = "host.docker.internal:9000"
s3_username = "admin"
s3_password = "admin123"
bucket_name = "faasnn-bucket"

In [84]:
class Neuron(threading.Thread):
    def __init__(self, parameters, kafka_broker, redis_url):
        threading.Thread.__init__(self)
        params = json.loads(parameters)
        self.layer_id = params["layer_id"]
        self.layer_id_num = int(params["layer_id_num"])
        self.neuron_id = int(params["neuron_id"])
        self.is_final_layer = False if params["is_final_layer"] == "0" else True
        self.weights = np.array(json.loads(params["weights"]), dtype=np.float64)
        self.bias = np.array(float(params["bias"]), dtype=np.float64)
        self.activation = None if self.is_final_layer else params["activation"]
        self.activation_func = None if self.is_final_layer else ACTIVATIONS.get(self.activation, relu)
        self.kafka_broker = kafka_broker
        redis_host, port = redis_url.split(":")
        self.redis_handler = RedisHandler(redis_host, int(port), 0, 2)
        self.producer = None
        

    def fetch_input(self, image_id):
        key = f"initial_data_{image_id}" if self.layer_id_num == 0 else f"{self.layer_id_num - 1}_{image_id}"
        return np.frombuffer(self.redis_handler.get(key), dtype=np.float64)

    def process_and_send(self, image_id, input_data):
        #output = dot_with_bias(input_data, self.weights, self.bias, self.activation, False)
        z = np.dot(input_data, self.weights) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        self.producer.send_with_key(str(image_id), f"{self.neuron_id}|{format(output, '.17g')}")

    def run(self):
        # Instantiate Kafka consumer and producer inside the thread.
        consumer_handler = KafkaConsumerHandler(f'layer-{self.layer_id_num}', self.kafka_broker, group_id=f"{self.neuron_id}_{self.layer_id_num}_group")
        consumer = consumer_handler.get_consumer()
        self.producer = KafkaProducerHandler(self.kafka_broker, f'layer-{self.layer_id_num}-streams')
        last_msg_time = time.time()
        while True:
            got_message = False
            for msg in consumer.consume(500, timeout=0.2):
                if msg.error():
                    consumer_handler.error_handling(msg)
                message = msg.value().decode('utf-8')
                got_message = True
                last_msg_time = time.time()
                image_id_str = message
                image_id = int(image_id_str)
                try:
                    input_data = self.fetch_input(image_id)
                    self.process_and_send(image_id, input_data)
                except Exception:
                    pass
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                self.producer.close()
                self.redis_handler.close()
                break


import threading
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

class NeuronOutput(threading.Thread):
    def __init__(
        self,
        s3_host,
        s3_username,
        s3_password,
        bucket_name="faasnn-bucket",
        prefix="predictions",
        batch_size=500,
        batch_timeout=60
    ):
        threading.Thread.__init__(self)
        self.s3_client = S3Client("host.docker.internal:9000", "admin", "admin123")
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.batch_size = batch_size
        self.batch_timeout = batch_timeout
        self.file_counter = 0
        self.lock = threading.Lock()
        # Initialize batching state
        self._reset_batch()

    def _reset_batch(self):
        self.csv_buffer = BytesIO()
        self.csv_buffer.write(b"image_id,prediction\n")  # CSV header directly as bytes
        self.record_count = 0
        self.last_batch_time = time.time()

    def run(self):
        consumer_handler = KafkaConsumerHandler('layer-output', KAFKA_BROKER, group_id="neuron_output_coord_group")
        consumer = consumer_handler.get_consumer()
        last_msg_time = time.time()

        while True:
            got_message = False
            for msg in consumer.consume(500, timeout=0.2):
                if msg.error():
                    consumer_handler.error_handling(msg)
                    continue

                image_id, prediction = msg.value().decode('utf-8').split("|")
                got_message = True
                last_msg_time = time.time()

                with self.lock:
                    line = f"{image_id},{prediction}\n".encode("utf-8")
                    self.csv_buffer.write(line)
                    self.record_count += 1
                    should_flush = (
                        self.record_count >= self.batch_size or
                        (time.time() - self.last_batch_time > self.batch_timeout)
                    )

                if should_flush:
                    self.flush_batch_async()

            if not got_message and (time.time() - last_msg_time > 15):
                consumer.commit()
                consumer.close()
                with self.lock:
                    if self.record_count > 0:
                        self.flush_batch_async()  # Flush remaining on shutdown
                break

    def flush_batch_async(self):
        with self.lock:
            if self.record_count == 0:
                return

            # Snapshot the buffer reference (BytesIO object)
            buffer_snapshot = self.csv_buffer
            file_name_snapshot = f"{self.prefix}/part-{self.file_counter:05d}.csv"
            self.file_counter += 1

            # Reset buffer and counters immediately
            self._reset_batch()

        # Submit background flush
        self.executor.submit(self.flush_batch_to_s3, buffer_snapshot, file_name_snapshot)

    def flush_batch_to_s3(self, buffer, file_name):
        try:
            buffer.seek(0)  # Rewind buffer to start before reading
            content = buffer.read()
            self.s3_client.put_object(self.bucket_name, file_name, content)
        except Exception as e:
            print(f"Error uploading {file_name} to S3: {e}")



class LayerOutput(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        # self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 8)
        self.last_layer_id_num = 1
        self.executor = ThreadPoolExecutor(max_workers=4)

    def run(self):
        consumer_handler = KafkaConsumerHandler(f'layer-output', KAFKA_BROKER, group_id=f"neuron_output_coord_group")
        consumer = consumer_handler.get_consumer()
        last_msg_time = time.time()
        while True:
            got_message = False
            for msg in consumer.consume(500, timeout=0.2):
                if msg.error():
                    consumer_handler.error_handling(msg)
                message = msg.value().decode('utf-8')
                got_message = True
                last_msg_time = time.time()
                image_id = int(message)
                try:
                    key = f"{self.last_layer_id_num}_{image_id}"
                    #outputs = np.frombuffer(self.redis_handler.get(key), dtype=np.float64)
                    #prediction = int(np.argmax(outputs))
                    #self.redis_handler.hset('streams:predictions', image_id, prediction)
                except Exception:
                    pass
            if not got_message and (time.time() - last_msg_time > 15):
                consumer.commit()
                consumer.close()
                # self.redis_handler.close()
                break

# Load network and dataset
data = json.load(open("neurons.json"))

neurons = []

for neuron in data:
    neurons += [Neuron(parameters=json.dumps(neuron), kafka_broker=KAFKA_BROKER, redis_url=redis_url)]

neuron_output = NeuronOutput(s3_host=s3_host, s3_username=s3_username, s3_password=s3_password)

# Start all threads
for thread in neurons:
    thread.start()

neuron_output.start()

print("Threads started")

Threads started


In [23]:
import redis

# Connect to Redis
r = RedisHandler('host.docker.internal', 6379, 0)

# Get hashes from Redis
images_label = r.hgetall('images_label')
predictions = r.hgetall('streams:predictions')

# Decode bytes to string
images_label = {k.decode(): v.decode() for k, v in images_label.items()}
predictions = {k.decode(): v.decode() for k, v in predictions.items()}

# Calculate accuracy
correct = 0
total = len(images_label)

for field, label_val in images_label.items():
    pred_val = predictions.get(field, None)
    if pred_val == label_val:
        correct += 1

accuracy = (correct / total) * 100 if total > 0 else 0

print(f'Accuracy: {accuracy:.2f}% ({correct}/{total})')


Accuracy: 95.30% (1906/2000)


In [9]:
# Wait for all threads to complete
for thread in neurons:
    thread.join()

neuron_output.join()

print("Threads finished")

Threads finished
